In [1]:
# https://www.i14y.admin.ch/api/index.html

In [1]:
import requests
from dict2xml import dict2xml
import json

In [12]:
def download_i14y_data():

    url = 'https://input.i14y.admin.ch/api/ConceptSummary/search?page=0&pageSize=1000'
    data = requests.get(url).json()

    codelists = []
    codes = [(entry['id'], entry['name']) for entry in data if entry['conceptType'] == 'CodeList']

    for id, name in codes:
        url = f'https://input.i14y.admin.ch/api/ConceptInput/{id}/codelistEntries?page=1&pageSize=10000'
        response = requests.get(url)
        if response.status_code == 200:
            codelists.append({'id': id, 'name': name, 'codelist': response.json()})

    # save_file = open("codelists.json", "w")  
    # json.dump(codelists, save_file, indent = 6)
    # save_file.close() 
            
    return codelists

In [64]:
def generate_isurvey_codelist(codelists):
    document = '''
        <?xml version="1.0" encoding="UTF-8"?>
        <document>
        <LimeSurveyDocType>Label set</LimeSurveyDocType>
        <DBVersion>623</DBVersion>
        '''


    labelsets = '''
    <labelsets>
    <fields>
    <fieldname>lid</fieldname>
    <fieldname>label_name</fieldname>
    <fieldname>languages</fieldname>
    </fields>
    <rows>
    '''

    labels = '''
    <labels>
    <fields>
    <fieldname>id</fieldname>
    <fieldname>lid</fieldname>
    <fieldname>code</fieldname>
    <fieldname>sortorder</fieldname>
    </fields>
    <rows>
    '''

    label_l10ns = '''
    <label_l10ns>
    <fields>
    <fieldname>id</fieldname>
    <fieldname>label_id</fieldname>
    <fieldname>title</fieldname>
    <fieldname>language</fieldname>
    </fields>
    <rows>
    '''

    lid = 1
    code_id = 1
    lang_id = 1
    languages = ['de', 'en', 'fr', 'it']

    for entry in codelists:

        labelsets += f''' 
            <row>
                <lid><![CDATA[{lid}]]></lid>
                <label_name><![CDATA[{entry['name']['de']}]]></label_name>
                <languages><![CDATA[de en fr it]]></languages>
            </row>
        '''
        for code in entry['codelist']:
            labels += f'''
            <row>
                <id><![CDATA[{code_id}]]></id>
                <lid><![CDATA[{lid}]]></lid>
                <code><![CDATA[{code['value']}]]></code>
                <sortorder><![CDATA[{code_id}]]></sortorder>
            </row>
            '''

            for i in range(len(languages)):
                label_l10ns += f'''
                    <row>
                        <id><![CDATA[{lang_id}]]></id>
                        <label_id><![CDATA[{code_id}]]></label_id>
                        <title><![CDATA[{code['name'][languages[i]]}]]></title>
                        <language><![CDATA[[{languages[i]}]]></language>
                    </row>
                '''

                lang_id += 1

            code_id += 1

        
        lid += 1

    labelsets += '''
    </rows>
    </labelsets
    '''

    labels += '''
    </rows>
    </labels>
    '''

    label_l10ns += '''
    </rows>
    </label_l10ns>
    '''

    document = labelsets + labels + label_l10ns

    return document

xml = generate_isurvey_codelist(codelists[:2])
print(xml)


    <labelsets>
    <fields>
    <fieldname>lid</fieldname>
    <fieldname>label_name</fieldname>
    <fieldname>languages</fieldname>
    </fields>
    <rows>
     
            <row>
                <lid><![CDATA[1]]></lid>
                <label_name><![CDATA[Rentenaufschub der 1. Säule]]></label_name>
                <languages><![CDATA[de en fr it]]></languages>
            </row>
         
            <row>
                <lid><![CDATA[2]]></lid>
                <label_name><![CDATA[Vorbezug der Rente der 1. Säule ]]></label_name>
                <languages><![CDATA[de en fr it]]></languages>
            </row>
        
    </rows>
    </labelsets
    
    <labels>
    <fields>
    <fieldname>id</fieldname>
    <fieldname>lid</fieldname>
    <fieldname>code</fieldname>
    <fieldname>sortorder</fieldname>
    </fields>
    <rows>
    
            <row>
                <id><![CDATA[1]]></id>
                <lid><![CDATA[1]]></lid>
                <code><![CDATA[0]]></code>
     

In [54]:
print(labelsets)


 <labelsets>
  <fields>
   <fieldname>lid</fieldname>
   <fieldname>label_name</fieldname>
   <fieldname>languages</fieldname>
  </fields>
  <rows>
 
        <row>
            <lid><![CDATA[1]]></lid>
            <label_name><![CDATA[Rentenaufschub der 1. Säule]]></label_name>
            <languages><![CDATA[de en fr it rm]]></languages>
        </row>
     
        <row>
            <lid><![CDATA[2]]></lid>
            <label_name><![CDATA[Vorbezug der Rente der 1. Säule ]]></label_name>
            <languages><![CDATA[de en fr it rm]]></languages>
        </row>
     
        <row>
            <lid><![CDATA[3]]></lid>
            <label_name><![CDATA[Lebend- oder Totgeburt]]></label_name>
            <languages><![CDATA[de en fr it rm]]></languages>
        </row>
     
        <row>
            <lid><![CDATA[4]]></lid>
            <label_name><![CDATA[Einlings- oder Mehrlingsgeburt]]></label_name>
            <languages><![CDATA[de en fr it rm]]></languages>
        </row>
     
  

In [36]:
codelist_index = 0
entry_index = 0
entry = codelists[codelist_index]['codelist'][entry_index]


def generate_label_list(codelist):
    languages = ['de', 'en', 'fr', 'it', 'rm']
    rows = []
    id = 0

    for entry in codelist:

        for j in range(len(languages)):
            name = entry['name'][languages[i]]

            if not name:
                name = entry['name'][languages[0]]

            row = {
                'row': {
                    'id': ud,
                    'label_id': entry['value'],
                    'title': f'{name}',
                    'language': f'{languages[i]}'
                    }
                }
            id += 1
            rows.append(row)

print(dict2xml(rows, indent='    '))

# print(codelists[codelist_index]['codelist'][entry_index])


    <row>
        <id>0</id>
        <label_id>0</label_id>
        <language>de</language>
        <title>Ohne Aufschub</title>
    </row>


    <row>
        <id>1</id>
        <label_id>0</label_id>
        <language>en</language>
        <title>Ohne Aufschub</title>
    </row>


    <row>
        <id>2</id>
        <label_id>0</label_id>
        <language>fr</language>
        <title>Sans ajournement</title>
    </row>


    <row>
        <id>3</id>
        <label_id>0</label_id>
        <language>it</language>
        <title>Senza rinvio</title>
    </row>


    <row>
        <id>4</id>
        <label_id>0</label_id>
        <language>rm</language>
        <title>Ohne Aufschub</title>
    </row>



In [10]:
data = {}
data['LimeSurveyDocType'] = 'Label set'
data['DBVersion'] = 623

c_lst = []
for codelist in codelists:
    

xml = dict2xml(data, wrap='document', indent='    ')
save_file = open("codelists.lsl", "w")  
save_file.write(xml)
save_file.close() 